In [1]:
import pygame
import math
pygame.init()

screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))

pygame 2.5.2 (SDL 2.28.3, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


MVP
- FLOOR
- WALLS
- PLAYERS
- Collision mechanics
- RESOURCE (INCL. PLATE)
- RESOURCE SOURCE
- CONVEYOR BELT
- CHOPPING BOARD
- FRYER

In [2]:
FLOOR_GRAPHIC = pygame.image.load('floor.png')
WALL_GRAPHIC = pygame.image.load('wall.png')
PLAYER1_GRAPHIC = pygame.image.load('player1.png')
PLAYER2_GRAPHIC = pygame.image.load('player2.png')
FISH_ICON = pygame.image.load('fish.png')
POTATO_ICON = pygame.image.load('potato.png')


PLAYER_SPEED = 2

In [3]:
SIZE = 32

class Object(pygame.sprite.Sprite):
    def __init__(self, position, graphic):
        super().__init__()
        self.image = graphic
        self.rect = self.image.get_rect(center=position)
        self.speed = PLAYER_SPEED
        
    
    def draw(image_file):
        image = pygame.image.load(image_file)
        
    def check_distance(self,object2):
        x1, y1 = self.rect.centerx, self.rect.centery
        x2, y2 = self.rect.centerx, self.rect.centery
        return math.sqrt((x2-x1)**2 + (y2-y1)**2)
        
Floors = pygame.sprite.Group()
class Floor(Object):
    def __init__(self, position):
        super().__init__(position, FLOOR_GRAPHIC)

Walls = pygame.sprite.Group()
class Wall(Object):
    def __init__(self, position):
        super().__init__(position, WALL_GRAPHIC)

In [4]:
Players = pygame.sprite.Group()
class Player(Object):
    def __init__(self, position, player_graphic, controls):
        super().__init__(position, player_graphic)
        self.controls = controls
        self.hands = None
        self.HAND_LENGTH = SIZE*math.sqrt(2)
        
    def update(self, keys):
        if keys[self.controls['ACTION']]:
            for source in Sources:
                self.grab_resource(source)
        
        self.moved = False
        prex, prey = self.rect.x, self.rect.y
        if keys[self.controls['UP']]:
            self.rect.y -= self.speed
            self.moved = True
            
        if keys[self.controls['DOWN']]:
            self.rect.y += self.speed
            self.moved = True
            
        if keys[self.controls['LEFT']]:
            self.rect.x -= self.speed
            self.moved = True
            
        if keys[self.controls['RIGHT']]:
            self.rect.x += self.speed
            self.moved = True
            
        self.last_move = (self.rect.x - prex, self.rect.y - prey)
        if(self.moved):
            self.check_collision()
            
        
    def bounce_back(self):
        self.rect.x, self.rect.y = self.rect.x - self.last_move[0], self.rect.y - self.last_move[1]
        
        
    def check_collision(self):
        collisions = pygame.sprite.spritecollide(self, Walls, False)
        if(collisions):
            self.bounce_back()
            
        players = 0
        for player in Players:
            print(self)
            print(player)
            if(self.rect.colliderect(player)):
                players += 1
                
        assert(players > 0)
        if(players > 1):
            self.bounce_back()
    
    def grab_resource(self, source):
        if (self.check_distance(source) < self.HAND_LENGTH):
            source.give_resource(self)
    


In [5]:
class ResourceSource(Object):
    def __init__(self, position, resource, graphic):
        super().__init__()
        self.image = graphic
        self.rect = self.image.get_rect(center=position)
        self.resource = resource
    
    
    # take the resource = player now has resource
    def give_resource(self, player):
        player.hands = self.resource(player)
        
    

In [ ]:
class Resource():
    def __init__(self, icon, place):
        self.icon = icon
        self.place = place
        
class Fish(Resource):
    def __init__(self, place):
        super().__init__(self, FISH_ICON, place)
        

class Potato(Resource):
    def __init__(self, place):
        super().__init__(self, POTATO_ICON, place)

In [6]:
ROWS = 12
COLS = 18

START_X, START_Y = (80, 60)
floor_plan_matrix = [([0]*COLS) for i in range(ROWS)]
for i in range (COLS):
    floor_plan_matrix[0][i] = 1
    floor_plan_matrix[ROWS-1][i] = 1

for i in range (ROWS):
    floor_plan_matrix[i][0] = 1
    floor_plan_matrix[i][COLS-1] = 1
    
WALL_COLUMN = 11
WALL_LENGTH = 8
for i in range (8):
    floor_plan_matrix[i][WALL_COLUMN] = 1


obj_types = [[Floor, Floors], [Wall, Walls]]
for i in range (ROWS):
    for j in range (COLS):
        obj_class, obj_sprite = obj_types[floor_plan_matrix[i][j]]
        object = obj_class([START_X+SIZE/2 + j*SIZE, START_Y+SIZE/2 + i*SIZE])
        obj_sprite.add(object)


player1_controls = {
    "UP": pygame.K_w,
    "DOWN": pygame.K_s,
    "LEFT": pygame.K_a,
    "RIGHT": pygame.K_d,
    "ACTION": pygame.K_e
}
player2_controls = {
    "UP": pygame.K_UP,
    "DOWN": pygame.K_DOWN,
    "LEFT": pygame.K_LEFT,
    "RIGHT": pygame.K_RIGHT,
    "ACTION": pygame.K_SPACE
}

Player1 = Player([200, 120], PLAYER1_GRAPHIC, player1_controls)
Players.add(Player1)

Player2 = Player([500, 120], PLAYER2_GRAPHIC, player2_controls)
Players.add(Player2)

In [7]:
running = True
clock = pygame.time.Clock()

print(Player1.rect.colliderect(Player1))

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill((255, 255, 255)) 

    Players.update(pygame.key.get_pressed())
    if pygame.QUIT in pygame.event.get():
        running = False
        
    Floors.draw(screen)
    Walls.draw(screen)
    Players.draw(screen)

    pygame.display.flip()
    clock.tick(60)

# Quit Pygame
pygame.quit()


True


KeyboardInterrupt: 